In [1]:
%%bash
# Go to folder containing the source code
cd /app/mialsuperresolutiontoolkit/
# Install the pymialsrtk package inside the python/conda environment
python setup.py install --force

running install
running build
running build_py
copying pymialsrtk/interfaces/preprocess.py -> build/lib/pymialsrtk/interfaces
running build_scripts
running install_lib
copying build/lib/pymialsrtk/info.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk
copying build/lib/pymialsrtk/interfaces/registration.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/reconstruction.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/preprocess.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/__init__.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/utils.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/__in

In [2]:
import os
import json

# Imports from nipype
from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline import Node, Workflow

# Import the implemented interface from pymialsrtk
import pymialsrtk.interfaces.preprocess as preprocess

# Set different variables (defined in cell 2) such that we do not have to rerun cell 2
bids_dir = os.path.join('/fetaldata')
output_dir = os.path.join('/fetaldata','derivatives','mialsrtk')
mask_dir = os.path.join('/fetaldata','derivatives','manual_masks')
subject = '01'

#Make sure we start from empty derivatives
import shutil
shutil.rmtree(output_dir)

#Check if mandatory derivatives dataset_description.json exists in derivatives/mialsrtk.
# If not, it is created before running the workflow, otherwise BIDSDataGrabber is not happy and raises an error. 

mialsrtk_dataset_description_json = os.path.join(output_dir,'dataset_description.json')

print('Check for {}'.format(mialsrtk_dataset_description_json))
if not os.access(mialsrtk_dataset_description_json, os.R_OK):
    data = {'PipelineDescription':{'Name': 'MIAL Super-Resolution ToolKit', 
                                'Version': 'v2.0.0-beta', 
                                'CodeURL': 'https://github.com/sebastientourbier/mialsuperresolutiontoolkit'
                                  },
            'Name': 'MIAL Super-Resolution ToolKit',
            'BIDSVersion': '1.2.0'
           }
    os.makedirs(output_dir)
    with open(mialsrtk_dataset_description_json, 'w+') as outfile:
        json.dump(data, outfile, indent=4)
    print('File {} was created'.format(mialsrtk_dataset_description_json))
else:
    print('File {} already exists'.format(mialsrtk_dataset_description_json))

# Construct the workflow
wf = Workflow(name="ist",base_dir=os.path.join(output_dir,'sub-{}'.format(subject),'nipype'))

bg = Node(interface=BIDSDataGrabber(infields = ['subject']),name='bids_grabber')
bg.inputs.base_dir = bids_dir
bg.inputs.subject = subject
bg.inputs.index_derivatives = True
bg.inputs.raise_on_empty = False
bg.inputs.output_query = {'T2ws': dict(suffix='T2w',datatype='anat',extensions=[".nii",".nii.gz"])}


# 
## Not specified maximum value
ist_nomax = Node(interface=preprocess.MultipleMialsrtkIntensityStandardization(),name='ist_nomax')
ist_nomax.inputs.bids_dir = bids_dir
                  
wf.connect(bg, "T2ws", ist_nomax, "input_images")

# 
## Specified maximum value
ist_max = Node(interface=preprocess.MultipleMialsrtkIntensityStandardization(),name='ist_max')
ist_max.inputs.bids_dir = bids_dir
ist_max.inputs.in_max = 271.0
                  
wf.connect(bg, "T2ws", ist_max, "input_images")


res = wf.run()

Check for /fetaldata/derivatives/mialsrtk/dataset_description.json
File /fetaldata/derivatives/mialsrtk/dataset_description.json was created
191016-16:48:13,285 nipype.workflow INFO:
	 Workflow ist settings: ['check', 'execution', 'logging', 'monitoring']
191016-16:48:13,288 nipype.workflow INFO:
	 Running serially.
191016-16:48:13,288 nipype.workflow INFO:
	 [Node] Setting-up "ist.bids_grabber" in "/fetaldata/derivatives/mialsrtk/sub-01/nipype/ist/bids_grabber".
191016-16:48:13,291 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")


/opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/bids/layout/layout.py:659: UserWarning: In pybids 0.9.0, the 'extensions' filter was deprecated in favor of 'extension'. The former will stop working in 0.11.0.
  warnings.warn("In pybids 0.9.0, the 'extensions' filter was "


191016-16:48:14,258 nipype.workflow INFO:
	 [Node] Finished "ist.bids_grabber".
191016-16:48:14,258 nipype.workflow INFO:
	 [Node] Setting-up "ist.ist_max" in "/fetaldata/derivatives/mialsrtk/sub-01/nipype/ist/ist_max".
191016-16:48:14,261 nipype.workflow INFO:
	 [Node] Running "ist_max" ("pymialsrtk.interfaces.preprocess.MultipleMialsrtkIntensityStandardization")
input_image /fetaldata/sub-01/anat/sub-01_run-1_T2w.nii.gz
... cmd: mialsrtkIntensityStandardization --input "/fetaldata/sub-01/anat/sub-01_run-1_T2w.nii.gz" --output "/fetaldata/derivatives/mialsrtk/sub-01/nipype/ist/ist_max/sub-01_run-1_T2w_ist.nii.gz" --max "271.0"
input_image /fetaldata/sub-01/anat/sub-01_run-2_T2w.nii.gz
... cmd: mialsrtkIntensityStandardization --input "/fetaldata/sub-01/anat/sub-01_run-2_T2w.nii.gz" --output "/fetaldata/derivatives/mialsrtk/sub-01/nipype/ist/ist_max/sub-01_run-2_T2w_ist.nii.gz" --max "271.0"
input_image /fetaldata/sub-01/anat/sub-01_run-3_T2w.nii.gz
... cmd: mialsrtkIntensityStandardiz